# VeriScore TUTORIAL
https://colab.research.google.com/drive/14cJsd5xu-paXb1ld72kF3WA97qzcyEn1?authuser=1#scrollTo=RunpVmK0K8tX open in provate window at EPFL
https://github.com/Yixiao-Song/VeriScore/tree/main

In [41]:
!which python3

/Users/dianaavalos/PycharmProjects/pythonProject/.venv/bin/python3


In [2]:
# # Let's install veriscore package and download en_core_web_sm
# !pip install --upgrade VeriScore
# !python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
/Users/dianaavalos/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 193.1 kB/s eta 0:00:0000:0100:02

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Environment variable setup

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.environ.get("OPENAI_API_KEY")
os.environ['SERPER_KEY_PRIVATE'] = 'd6ce1e6b04b30af4062b828af905df9f84980b19'
# https://serper.dev/api-key

## Data prep
The data format should be JSONL, and each JSON object should contain keys for `question`, `response`, `prompt_source`, and the `model` that generates the response.

You can get sample data from [here](https://github.com/Yixiao-Song/VeriScore/tree/main/data).

In [1]:
import json
file_path =  "test_veriscore/data_sample.jsonl"
test_input_file = "test_veriscore/data_sample.jsonl"

with open(test_input_file, "r") as f:
    first_line = json.loads(f.readline())
    print(first_line.keys())

print(json.dumps(first_line)[:500])  

# dict_keys(['question', 'prompt_source', 'response', 'prompt_tok_cnt', 'response_tok_cnt', 'model', 'abstained', 'claim_list', 'all_claim_lst', 'claim_search_results', 'claim_verification_result'])
print(first_line['response'][:100])  # Replace 'question' with any key


dict_keys(['question', 'prompt_source', 'response', 'prompt_tok_cnt', 'response_tok_cnt', 'model', 'abstained', 'claim_list', 'all_claim_lst', 'claim_search_results', 'claim_verification_result'])
{"question": "I am a 21 year old girl and got my period twice in the same month. this is the first time this happened, why might it be the case.", "prompt_source": "ShareGPT", "response": "Experiencing two periods in one month can be concerning, but there are several potential reasons why this might happen. Here are some common causes:\n\n1. **Hormonal Imbalance**: Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles. This can be due to stress, changes 
Experiencing two periods in one month can be concerning, but there are several potential reasons why


prep input file by emptying keys

In [45]:
# TO DO : adapt the input file because we want to get rid of these 2 params in input prompts: first_line['claim_search_results'] and first_line['claim_verification_result']

# Keys to empty
keys_to_empty = [
    "prompt_tok_cnt","response_tok_cnt", "claim_list", "all_claim_lst", 
    "claim_search_results", "claim_verification_result"
]

# Empty the specified keys
for key in keys_to_empty:
    first_line[key] = None if isinstance(first_line[key], (int, bool)) else []


In [46]:
# 2 - Write each dictionary as a new line in JSONL format
test_input_file = "test_veriscore/data_sample_2.jsonl"

with open(f"{test_input_file}", "w", encoding="utf-8") as jsonl_file:
    jsonl_file.write(json.dumps(first_line) + "\n")

# Run Veriscore for Claim extraction

In [36]:
# Run claim extraction using prompting method with gpt-4 model
# !python3 -m veriscore.extract_claims --model_name gpt-4-0125-preview --data_dir ./test --input_file data_sample.jsonl --output_dir ./test

model='gpt-4o-mini'
output_dir=""
input_file="test_veriscore/data_sample_2.jsonl"
!python3 -m veriscore.extract_claims --model_name $model --data_dir ./test_veriscore --input_file data_sample_2.jsonl --output_dir ./test_veriscore/output


/Users/dianaavalos/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  6.02it/s]
extracted claims are saved at ./test_veriscore/output/claims_data_sample_2.jsonl


In [ ]:
input_file="/Users/dianaavalos/Desktop/VeriScore-main/data/data_sample.jsonl"

In [48]:
# check results
output_file="test_veriscore/output/claims_data_sample_2.jsonl"
with open(output_file, "r") as f:
    first_line = json.loads(f.readline().strip())  # Read only the first line and parse it
    # print(first_line)
    print(first_line['claim_list'][:10])  # Replace 'question' with any key


[[None], [None], ['Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.'], [None], ["Starting, stopping, or changing a method of birth control can affect a person's menstrual cycle."], ['Hormonal contraceptives can cause irregular bleeding.'], ['High levels of stress can impact the menstrual cycle by affecting the hormones that regulate it.'], ['Polycystic Ovary Syndrome (PCOS) is a common condition.', 'Polycystic Ovary Syndrome (PCOS) can cause irregular periods.', 'Polycystic Ovary Syndrome (PCOS) can cause more than one period in a month.'], ['Hyperthyroidism can affect the menstrual cycle.', 'Hypothyroidism can affect the menstrual cycle.'], ['Hyperthyroidism can affect the menstrual cycle.', 'Hypothyroidism can affect the menstrual cycle.']]


In [55]:
# initialize objects
from veriscore.claim_extractor import ClaimExtractor
model_name = first_line["model"]
model_name = 'gpt-4o-mini'
use_external_model = 'store_true'
use_external_model = True
cache_dir = './data/cache'
data_dir = "./test_veriscore"
input_file = "data_sample_2.jsonl"
output_dir ="./test_veriscore/output"

claim_extractor = ClaimExtractor(model_name, cache_dir, use_external_model=False)
# if True need to install pytorch, if False, use OpenAI 


Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...


In [56]:
response = first_line["response"]
prompt_source = first_line["prompt_source"]

snippet_lst, claim_list, all_claims, prompt_tok_cnt, response_tok_cnt = claim_extractor.non_qa_scanner_extractor(response) 

Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data/cache/gpt-4o-mini/claim_extraction_cache.json...
Loading cache from ./data

In [58]:
all_claims

['Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.',
 'Changes in birth control methods can affect menstrual cycles.',
 'Hormonal contraceptives can cause irregular bleeding.',
 'High levels of stress can impact the menstrual cycle.',
 'Stress can affect the hormones that regulate the menstrual cycle.',
 'Polycystic Ovary Syndrome (PCOS) is a common condition.',
 'PCOS can cause irregular periods.',
 'PCOS can include having more than one period in a month.',
 'Hyperthyroidism can affect the menstrual cycle.',
 'Hypothyroidism can affect the menstrual cycle.',
 'Uterine fibroids are non-cancerous growths in the uterus.',
 'Uterine polyps are non-cancerous growths in the uterus.',
 'Uterine fibroids can cause irregular bleeding.',
 'Uterine polyps can cause irregular bleeding.',
 'Pelvic inflammatory disease (PID) can cause irregular bleeding.',
 'Other infections can cause irregular bleeding.',
 'Some women can start experiencing perimeno

In [1]:
# 1 -create dictionary with keys : 
dict_keys ={}

dict_keys(['prompt_source', 'response', 'prompt_tok_cnt', 'response_tok_cnt', 'model', 'abstained', 'claim_list', 'all_claim_lst', 'claim_sear ch_results', 'claim_verification_result'])
# 2 - fill dict_keys['response'] with chunk
# 3 run in veriscore to see what happens


NameError: name 'dict_keys' is not defined

In [52]:
# check results
output_file="test_veriscore/output/claims_data_sample.jsonl"
with open(output_file, "r") as f:
    first_line = json.loads(f.readline().strip())  # Read only the first line and parse it
    print(first_line)
    # print(first_line.keys())

## Also works: 
# with open(output_file, "r") as f:
#     for line in f:  # Read each line separately
#         json_obj = json.loads(line.strip())  # Convert each line to a dictionary
#         print(json_obj)

{'question': 'I am a 21 year old girl and got my period twice in the same month. this is the first time this happened, why might it be the case.', 'prompt_source': 'ShareGPT', 'response': 'Experiencing two periods in one month can be concerning, but there are several potential reasons why this might happen. Here are some common causes:\n\n1. **Hormonal Imbalance**: Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles. This can be due to stress, changes in weight, or other underlying health conditions.\n\n2. **Birth Control**: If you have recently started, stopped, or changed your method of birth control, it can affect your menstrual cycle. Hormonal contraceptives, in particular, can cause irregular bleeding.\n\n3. **Stress**: High levels of stress can impact your menstrual cycle by affecting the hormones that regulate it.\n\n4. **Polycystic Ovary Syndrome (PCOS)**: PCOS is a common condition that can cause irregular periods, including having m

# Decomposing  steps in veriscore.extract_claims
now that we have run veriscore.extract_claims, let's do it without their package to understand it!

## Initialize objects
in https://github.com/Yixiao-Song/VeriScore/blob/main/veriscore/extract_claims.py:

In [54]:
# Load inputs:
data_dir = "./test_veriscore"
input_file = "data_sample.jsonl"
input_path = os.path.join(data_dir, input_file)
model_name = 'gpt-4o-mini'
output_dir = "./test_veriscore/output"
use_external_model = 'store_true'
cache_dir = './data/cache'


with open(input_path, "r") as f:
    data = [json.loads(x) for x in f.readlines() if x.strip()]


In [60]:
from veriscore.claim_extractor import ClaimExtractor

# Run Veriscore for Evidence Retrieval

Before verifying claims, you need to obtain search results as evidence for each claim. We use the SERPER API to fetch results from Google.

In [ ]:
# !python3 -m veriscore.extract_claims --model_name $model --data_dir ./test_veriscore --input_file data_sample_2.jsonl --output_dir ./test_veriscore/output


In [60]:
!python3 -m veriscore.retrieve_evidence --data_dir /content/sample_data --input_file claims_data_sample.jsonl --output_dir /content/sample_data

/Users/dianaavalos/PycharmProjects/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "veriscore/retrieve_evidence.py", line 28, in <module>
    with open(input_path, "r") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/claims_data_sample.jsonl'


In [67]:
input_data_sample = "/Users/dianaavalos/Desktop/VeriScore-main/data/data_sample.jsonl"
with open(f"{input_data_sample}", "r", encoding="utf-8") as jsonl_file:
    first_line = json.loads(jsonl_file.readline().strip())  # Read and parse first line
    # input_data_sample_txt = [json.loads(line) for line in jsonl_file]  # Convert each line to a dictionary


In [68]:
first_line

{'question': 'I am a 21 year old girl and got my period twice in the same month. this is the first time this happened, why might it be the case.',
 'prompt_source': 'ShareGPT',
 'response': 'Experiencing two periods in one month can be concerning, but there are several potential reasons why this might happen. Here are some common causes:\n\n1. **Hormonal Imbalance**: Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles. This can be due to stress, changes in weight, or other underlying health conditions.\n\n2. **Birth Control**: If you have recently started, stopped, or changed your method of birth control, it can affect your menstrual cycle. Hormonal contraceptives, in particular, can cause irregular bleeding.\n\n3. **Stress**: High levels of stress can impact your menstrual cycle by affecting the hormones that regulate it.\n\n4. **Polycystic Ovary Syndrome (PCOS)**: PCOS is a common condition that can cause irregular periods, including having

In [71]:
from veriscore.search_API import SearchAPI  # Use the full package name
fetch_search = SearchAPI()

In [72]:
first_line['all_claim_lst']

['Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.',
 'Stress can cause irregular menstrual cycles.',
 'Changes in weight can cause irregular menstrual cycles.',
 'Underlying health conditions can cause irregular menstrual cycles.',
 'Starting, stopping, or changing the method of birth control can affect the menstrual cycle.',
 'Hormonal contraceptives can cause irregular bleeding.',
 'High levels of stress can impact the menstrual cycle by affecting the hormones that regulate it.',
 'Polycystic Ovary Syndrome (PCOS) can cause irregular periods.',
 'Polycystic Ovary Syndrome (PCOS) can cause more than one period in a month.',
 'Hyperthyroidism can affect the menstrual cycle.',
 'Hypothyroidism can affect the menstrual cycle.',
 'Uterine fibroids are non-cancerous growths in the uterus.',
 'Polyps in the uterus are non-cancerous growths.',
 'Uterine fibroids or polyps can cause irregular bleeding.',
 'Pelvic inflammatory disease (PID) can 

In [73]:
claim_snippets = fetch_search.get_snippets(first_line['all_claim_lst'])

Saving search cache ...
Loading cache ...
Saving search cache ...


In [74]:
claim_snippets

{'Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.': [{'title': 'Hormonal Imbalance: Causes, Symptoms & Treatment',
   'snippet': 'Specific hormone-related conditions that cause irregular periods include polycystic ovary syndrome (PCOS) and amenorrhea.',
   'link': 'https://my.clevelandclinic.org/health/diseases/22673-hormonal-imbalance'},
  {'title': 'Perimenopause - Symptoms and causes - Mayo Clinic',
   'snippet': 'The level of estrogen — the main female hormone — in your body rises and falls unevenly during perimenopause. Your menstrual cycles may lengthen ...',
   'link': 'https://www.mayoclinic.org/diseases-conditions/perimenopause/symptoms-causes/syc-20354666'},
  {'title': 'Hormone Imbalance and the Menstrual Cycle: Causes and Treatments',
   'snippet': 'An imbalance can arise if you have too much or too little of one or more hormones. The body requires precise levels of hormones to function properly.',
   'link': 'https://www.hea

In [75]:
first_line['all_claim_lst'][0]

'Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.'

In [85]:
first_line.keys()
first_line['claim_verification_result'][0]


{'claim': 'Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.',
 'search_results': 'Search result 1\nTitle: Hormonal Imbalance: Causes, Symptoms & Treatment\nLink: https://my.clevelandclinic.org/health/diseases/22673-hormonal-imbalance\nContent: Specific hormone-related conditions that cause irregular periods include polycystic ovary syndrome (PCOS) and amenorrhea.\n\nSearch result 2\nTitle: Hormonal imbalance: Symptoms, causes, and treatment\nLink: https://www.medicalnewstoday.com/articles/321486\nContent: Hormonal imbalances occur when there is too much or too little of a hormone in the blood. Common symptoms include weight changes, lower sex drive, and acne, ...\n\nSearch result 3\nTitle: Perimenopause - Symptoms and causes - Mayo Clinic\nLink: https://www.mayoclinic.org/diseases-conditions/perimenopause/symptoms-causes/syc-20354666\nContent: Irregular periods. As ovulation becomes more unpredictable, the length of time between periods m

In [76]:
claim_snippets_0 = fetch_search.get_snippets(first_line['all_claim_lst'][0])
claim_snippets_0 # does not work... answer is not logic

Loading cache ...
Saving search cache ...
Loading cache ...
Saving search cache ...


{'F': [{'title': 'Facebook - log in or sign up',
   'snippet': 'Create an account or log into Facebook. Connect with friends, family and other people you know. Share photos and videos, send messages and get updates.',
   'link': 'https://www.facebook.com/'},
  {'title': 'Ford Motor Company (F) Stock Price, News, Quote & History',
   'snippet': 'Ford Motor Company develops, delivers, and services Ford trucks, sport utility vehicles, commercial vans and cars, and Lincoln luxury vehicles worldwide.',
   'link': 'https://finance.yahoo.com/quote/F/'},
  {'title': 'Fred Oliveira (@f) / X',
   'snippet': "Worried about computers. AI ∩ Data ∩ VC at @capitalfactory. Before: @gumroad, @techcrunch, @webreakstuff. Co-author of Redis Cookbook for O'Reilly.",
   'link': 'https://x.com/f?lang=en'},
  {'title': 'Ford® - New Hybrid & Electric Vehicles, SUVs, Crossovers, Trucks ...',
   'snippet': 'Ford® is Built for America. Discover the latest lineup in new Ford vehicles! Explore hybrid & electric veh

# Claim Verification
Once we have obtained the search results for the claims, we can proceed to verify the claims using the retrieved evidence. This process provides:

1. **Label**: Indicates whether each claim can be supported or not.
   - You can modify the labeling with the `label_n` input option (default value is `3v):
     - `2` (binary): Supported / Unsupported
     - `3` (trinary): Supported / Contradicted / Inconclusive

2. **VeriScore**: Calculated as F1@K, where K is the median number of extracted facts.

You can limit the number of evidences used to verify a claim with the `search_res_num` option.

For the prompt-based approach, you need to set `data_dir/demos/` with [few-shot examples](https://drive.google.com/file/d/1Jyj11uFjS9a3X80CrO-LFcb41nxJaPdZ/view?usp=drive_link).

In [ ]:
# !python3 -m veriscore.verify_claims --model_name SYX/llama3_based_claim_verifier --data_dir ./data --input_file evidence_claims_data_sample.jsonl --output_dir ./data --label_n 2 --search_res_num 3 # Note: default search_res_num is 10.


In [4]:
import os
# TODO add key
import json
input_data_sample = "/Users/dianaavalos/Desktop/VeriScore-main/data/data_sample.jsonl"
with open(f"{input_data_sample}", "r", encoding="utf-8") as jsonl_file:
    first_line = json.loads(jsonl_file.readline().strip())  # Read and parse first line

In [5]:
# initialize objects
model_name = 'SYX/llama3_based_claim_verifier'
model_name = 'gpt-4o-mini'
cache_dir = './data/cache'
label_n = 3
demon_dir="/Users/dianaavalos/Desktop/VeriScore-main/data/demos"
use_external_model=False
search_res_num = 3

In [6]:
from veriscore.claim_verifier import ClaimVerifier
claim_verifier = ClaimVerifier(model_name=model_name, label_n=label_n,
                               cache_dir=cache_dir, demon_dir=demon_dir,
                               use_external_model=use_external_model)

In [9]:
claim_search_results = first_line["claim_search_results"]
claim_search_results

{'Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.': [{'title': 'Hormonal Imbalance: Causes, Symptoms & Treatment',
   'snippet': 'Specific hormone-related conditions that cause irregular periods include polycystic ovary syndrome (PCOS) and amenorrhea.',
   'link': 'https://my.clevelandclinic.org/health/diseases/22673-hormonal-imbalance'},
  {'title': 'Hormonal imbalance: Symptoms, causes, and treatment',
   'snippet': 'Hormonal imbalances occur when there is too much or too little of a hormone in the blood. Common symptoms include weight changes, lower sex drive, and acne, ...',
   'link': 'https://www.medicalnewstoday.com/articles/321486'},
  {'title': 'Perimenopause - Symptoms and causes - Mayo Clinic',
   'snippet': 'Irregular periods. As ovulation becomes more unpredictable, the length of time between periods may be longer or shorter, your flow may be light ...',
   'link': 'https://www.mayoclinic.org/diseases-conditions/perimenopaus

In [23]:
print(type(claim_search_results))
if isinstance(claim_search_results, dict):
    print("Keys:", claim_search_results.keys())

IndentationError: expected an indented block (998567564.py, line 5)

In [25]:
key_1 = next(iter(claim_search_results)) 
key_1
claim_search_results_1 = {key_1: [claim_search_results[key_1][0]]}
claim_search_results_1

{'Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.': [{'title': 'Hormonal Imbalance: Causes, Symptoms & Treatment',
   'snippet': 'Specific hormone-related conditions that cause irregular periods include polycystic ovary syndrome (PCOS) and amenorrhea.',
   'link': 'https://my.clevelandclinic.org/health/diseases/22673-hormonal-imbalance'}]}

In [13]:
print("Calling verifying_claim:")
print(claim_verifier.verifying_claim)

claim_verify_res_dict, prompt_tok_cnt, response_tok_cnt = claim_verifier.verifying_claim(
    claim_search_results, search_res_num=search_res_num)

Calling verifying_claim:
<bound method ClaimVerifier.verifying_claim of <veriscore.claim_verifier.ClaimVerifier object at 0x10be7df40>>
Calling verifying_claim...
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Inconclusive.###
not self.model
response: ###Supported.###
not self.model
response: ###Inconclusive.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Supported.###
not self.model
response: ###Suppo

In [26]:
claim_verify_res_dict_1, prompt_tok_cnt, response_tok_cnt = claim_verifier.verifying_claim(
    claim_search_results_1, search_res_num=search_res_num)

Calling verifying_claim...
not self.model
Loading cache from ./data/cache/gpt-4o-mini/claim_verification_cache.json...
response: ###Inconclusive.###


In [27]:
claim_verify_res_dict

[{'claim': 'Fluctuations in hormones such as estrogen and progesterone can cause irregular menstrual cycles.',
  'search_results': 'Search result 1\nTitle: Hormonal Imbalance: Causes, Symptoms & Treatment\nLink: https://my.clevelandclinic.org/health/diseases/22673-hormonal-imbalance\nContent: Specific hormone-related conditions that cause irregular periods include polycystic ovary syndrome (PCOS) and amenorrhea.\n\nSearch result 2\nTitle: Hormonal imbalance: Symptoms, causes, and treatment\nLink: https://www.medicalnewstoday.com/articles/321486\nContent: Hormonal imbalances occur when there is too much or too little of a hormone in the blood. Common symptoms include weight changes, lower sex drive, and acne, ...\n\nSearch result 3\nTitle: Perimenopause - Symptoms and causes - Mayo Clinic\nLink: https://www.mayoclinic.org/diseases-conditions/perimenopause/symptoms-causes/syc-20354666\nContent: Irregular periods. As ovulation becomes more unpredictable, the length of time between periods

In [90]:
import inspect
print(inspect.getfile(claim_verifier.verifying_claim))

veriscore/claim_verifier.py
